In [165]:
import geopandas as gpd
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)


In [166]:
year = 2020
election = "general"

In [167]:
df = pd.read_csv(f"../data/United States/{year}_{election}_precinct_general.csv", encoding="ISO-8859-1")
df = df.rename(columns={"state_postal": "state_po", "jurisdiction": "jurisdiction_name", "party_simplified": "party"})
df["candidate"] = df["candidate"].str.upper().str.strip()
df["state"] = df["state"].str.upper().str.strip()
df = df.dropna(subset=["party"])

df["candidate"] = df["candidate"].replace({
    "MORAN, JERRY": "JERRY MORAN",
    "GARRARD, ROBERT D.": "ROBERT D. GARRARD",
    "WIESNER, PATRICK": "PATRICK WIESNER",
    "ROBERT GARRARD": "ROBERT D. GARRARD"
})

if election == "senate" and year == 2016:
    df["party"] = df.apply(lambda x: f"{x['party']} ({x['candidate']})", axis=1)

C:\Users\cnava\AppData\Local\Temp\ipykernel_25196\60901638.py:1: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"../data/United States/{year}_{election}_precinct_general.csv", encoding="ISO-8859-1")


In [168]:
df.head()

,precinct,office,party_detailed,party,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,candidate,district,dataverse,year,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude
3,002,US PRESIDENT,DEMOCRAT,DEMOCRAT,TOTAL,3045,CLACKAMAS,41005.0,CLACKAMAS,41005.0,JOSEPH R BIDEN,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1
4,002,US PRESIDENT,LIBERTARIAN,LIBERTARIAN,TOTAL,94,CLACKAMAS,41005.0,CLACKAMAS,41005.0,JO JORGENSEN,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1
5,002,US PRESIDENT,OREGON PROGRESSIVE,OTHER,TOTAL,7,CLACKAMAS,41005.0,CLACKAMAS,41005.0,DARIO HUNTER,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1
6,002,US PRESIDENT,PACIFIC GREEN,OTHER,TOTAL,29,CLACKAMAS,41005.0,CLACKAMAS,41005.0,HOWIE HAWKINS,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1
7,002,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,2007,CLACKAMAS,41005.0,CLACKAMAS,41005.0,DONALD J TRUMP,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1


In [169]:
df[(df["state_po"] == "IA")].groupby("candidate").agg({"votes": "sum"})

,votes
candidate,
BROCK PIERCE,544
DON BLANKENSHIP,1707
DONALD J TRUMP,897672
HOWIE HAWKINS,3075
JO JORGENSEN,19637
JOSEPH R BIDEN,759061
KANYE WEST,3210
RICKI SUE KING,546
"ROCQUE ""ROCKY"" DE LA FUENTE",1082


In [170]:
df[(df["candidate"] == "STRAIGHT TICKET") & (df["party"] == "democratic")].party.unique()

array([], dtype=object)

In [171]:
if year == 2016 and election == "general":
    df.loc[(df["candidate"] == "STRAIGHT TICKET") & (df["party"] == "democratic"), "candidate"] = "HILLARY CLINTON"
    df.loc[(df["candidate"] == "STRAIGHT TICKET") & (df["party"] == "republican"), "candidate"] = "DONALD TRUMP"
    df.loc[(df["candidate"] == "STRAIGHT TICKET") & (df["party"] == "libertarian"), "candidate"] = "GARY JOHNSON"

In [172]:
df[df["candidate"]=="Straight ticket"].party.unique()

array([], dtype=object)

In [173]:
def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

In [174]:
df["polling_id"] = create_polling_id(df, ["state_po", "county_fips", "jurisdiction_name", "district", "precinct"])

In [175]:
df.head()

,precinct,office,party_detailed,party,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,candidate,district,dataverse,year,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude,polling_id
3,002,US PRESIDENT,DEMOCRAT,DEMOCRAT,TOTAL,3045,CLACKAMAS,41005.0,CLACKAMAS,41005.0,JOSEPH R BIDEN,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1,OR-41005.0-CLACKAMAS-STATEWIDE-002
4,002,US PRESIDENT,LIBERTARIAN,LIBERTARIAN,TOTAL,94,CLACKAMAS,41005.0,CLACKAMAS,41005.0,JO JORGENSEN,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1,OR-41005.0-CLACKAMAS-STATEWIDE-002
5,002,US PRESIDENT,OREGON PROGRESSIVE,OTHER,TOTAL,7,CLACKAMAS,41005.0,CLACKAMAS,41005.0,DARIO HUNTER,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1,OR-41005.0-CLACKAMAS-STATEWIDE-002
6,002,US PRESIDENT,PACIFIC GREEN,OTHER,TOTAL,29,CLACKAMAS,41005.0,CLACKAMAS,41005.0,HOWIE HAWKINS,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1,OR-41005.0-CLACKAMAS-STATEWIDE-002
7,002,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,2007,CLACKAMAS,41005.0,CLACKAMAS,41005.0,DONALD J TRUMP,STATEWIDE,PRESIDENT,2020,GEN,OREGON,False,False,OR,41,92,72,2020-11-03,False,1,OR-41005.0-CLACKAMAS-STATEWIDE-002


In [176]:
if election in ["senate", "representatives"]:
    df = df.rename(columns={"candidate": "candidate_name", "party_simplified": "candidate", "party": "candidate"})

In [177]:
if election in ["senate", "representatives"]:
    if year == 2020:
        df.loc[(df["state"] == "MINNESOTA") & (df["party_detailed"] == "DEMOCRATIC FARMER LABOR"), "candidate"] = "DEMOCRAT"
    # elif year == 2016:
    #     df.loc[(df["state"] == "MINNESOTA") & (df["party"] == "DEMOCRATIC FARMER LABOR"), "candidate"] = "DEMOCRAT"

In [178]:
tt = df.groupby(["polling_id", "candidate"]).agg({"votes": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate", "votes"]]

tt["rank"] = tt.groupby(["polling_id"])["votes"].rank("min", ascending=False).astype(int)

In [179]:
tt.head()

,polling_id,candidate,rate,votes,rank
0,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,BROCK PIERCE,0.003968,3,4
1,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,DON BLANKENSHIP,0.003968,3,4
2,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,DONALD J TRUMP,0.616402,466,1
3,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,"JAMES G ""JESSE VENTURA"" JANOS",0.002646,2,6
4,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,JO JORGENSEN,0.056878,43,3


In [180]:
df = pd.merge(df, tt, on=["polling_id", "candidate"])

In [181]:
tt[tt["polling_id"].str[:2] == "MN"]

,polling_id,candidate,rate,votes,rank
753899,MN-27001.0-AITKIN-STATEWIDE-AITKIN,ALYSON KENNEDY,0.000000,0,8
753900,MN-27001.0-AITKIN-STATEWIDE-AITKIN,BROCK PIERCE,0.000917,1,7
753901,MN-27001.0-AITKIN-STATEWIDE-AITKIN,DONALD J TRUMP,0.573786,626,1
753902,MN-27001.0-AITKIN-STATEWIDE-AITKIN,GLORIA LA RIVA,0.000000,0,8
753903,MN-27001.0-AITKIN-STATEWIDE-AITKIN,HOWIE HAWKINS,0.001833,2,6
...,...,...,...,...,...
790884,MN-27173.0-YELLOW MEDICINE-STATEWIDE-WOOD LAKE...,HOWIE HAWKINS,0.000000,0,4
790885,MN-27173.0-YELLOW MEDICINE-STATEWIDE-WOOD LAKE...,JO JORGENSEN,0.007463,1,3
790886,MN-27173.0-YELLOW MEDICINE-STATEWIDE-WOOD LAKE...,JOSEPH R BIDEN,0.223881,30,2
790887,MN-27173.0-YELLOW MEDICINE-STATEWIDE-WOOD LAKE...,KANYE WEST,0.000000,0,4


In [182]:
df_filtered = tt[["polling_id", "candidate", "votes", "rank", "rate"]].copy()
df_filtered = df_filtered.rename(columns={"votes": "value"})
df_filtered["candidate"] = df_filtered["candidate"].apply(lambda x: x.strip())

df_filtered.to_csv(f"../data_output/United States/{year}_{election}.csv.gz", compression="gzip", index=False)

In [183]:
df_filtered.head()

,polling_id,candidate,value,rank,rate
0,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,BROCK PIERCE,3,4,0.003968
1,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,DON BLANKENSHIP,3,4,0.003968
2,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,DONALD J TRUMP,466,1,0.616402
3,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,"JAMES G ""JESSE VENTURA"" JANOS",2,6,0.002646
4,AK-###-DISTRICT 1-STATEWIDE-01-446 AURORA,JO JORGENSEN,43,3,0.056878


In [184]:
df_location = df[["polling_id", "state", "state_po", "state_fips", "jurisdiction_name", 
"county_name", "county_fips", "district", "precinct"]].drop_duplicates() #"jurisdiction_fips", 

df_location.to_csv(f"../data_output/United States/{year}_{election}_location.csv.gz", compression="gzip", index=False)